In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml


import warnings
warnings.filterwarnings('ignore')

In [2]:
config = f"""
dataset: 
    base_path: /kaggle/input/child-mind-institute-detect-sleep-states

seed: 46
"""

CFG = yaml.load(config, Loader=yaml.SafeLoader)

In [3]:
train_events = pd.read_csv(f'{CFG["dataset"]["base_path"]}/train_events.csv')
train_events

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400
...,...,...,...,...,...
14505,fe90110788d2,33,wakeup,560604.0,2017-09-06T04:07:00-0400
14506,fe90110788d2,34,onset,574620.0,2017-09-06T23:35:00-0400
14507,fe90110788d2,34,wakeup,581604.0,2017-09-07T09:17:00-0400
14508,fe90110788d2,35,onset,NaN,NaN


In [4]:
# kfold
from sklearn.model_selection import GroupKFold

train_events['fold'] = -1
skf = GroupKFold(n_splits=5)
for fold, (train_idx, valid_idx) in enumerate(skf.split(train_events, train_events['series_id'], train_events['series_id'])):
    train_events.loc[valid_idx, 'fold'] = fold
train_events.groupby('fold')['series_id'].nunique()

fold
0    55
1    56
2    56
3    55
4    55
Name: series_id, dtype: int64

In [5]:
train_events

,series_id,night,event,step,timestamp,fold
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400,2
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400,2
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400,2
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400,2
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400,2
...,...,...,...,...,...,...
14505,fe90110788d2,33,wakeup,560604.0,2017-09-06T04:07:00-0400,2
14506,fe90110788d2,34,onset,574620.0,2017-09-06T23:35:00-0400,2
14507,fe90110788d2,34,wakeup,581604.0,2017-09-07T09:17:00-0400,2
14508,fe90110788d2,35,onset,NaN,NaN,2


In [6]:
train_events[["series_id", "fold"]].drop_duplicates().to_csv("train_folds.csv", index=False)